In [1]:
%run dataset.ipynb

In [2]:
# The office31 dataset
class Office31Dataset(Dataset):
    @property
    def base(self):
        return super(Office31Dataset, self)

In [3]:
# Initialize object
def office31_init(self, resolution=[100, 100], input_shape=[-1]):
    self.base.__init__('office31', 'dual_select')

    path = "office31"
    domain_names = list_dir(path)

    images = []
    didxs, oidxs = [], []

    for dx, dname in enumerate(domain_names):
        domainpath = os.path.join(path, dname, 'images')
        object_names = list_dir(domainpath)
        for ox, oname in enumerate(object_names):
            objectpath = os.path.join(domainpath, oname)
            filenames = list_dir(objectpath)
            for fname in filenames:
                if fname[-4:] != '.jpg':
                    continue
                imagepath = os.path.join(objectpath, fname)
                pixels = load_image_pixels(imagepath, resolution, input_shape)
                images.append(pixels)
                didxs.append(dx)
                oidxs.append(ox)

    self.image_shape = resolution + [3]

    xs = np.asarray(images, np.float32)

    ys0 = onehot(didxs, len(domain_names))
    ys1 = onehot(oidxs, len(object_names))
    ys = np.hstack([ys0, ys1])

    self.shuffle_data(xs, ys, 0.8)
    self.target_names = [domain_names, object_names]
    self.cnts = [len(domain_names)]

In [1]:
# Do postprocessing for forward computations
def office31_forward_postproc(self, output, y):
    outputs, ys = np.hsplit(output, self.cnts), np.hsplit(y, self.cnts)

    loss0, aux0 = self.base.forward_postproc(outputs[0], ys[0], 'select')
    loss1, aux1 = self.base.forward_postproc(outputs[1], ys[1], 'select')

    return loss0 + loss1, [aux0, aux1]

In [2]:
# Do postprocessing for back-propagations
def office31_backprop_postproc(self, G_loss, aux):
    aux0, aux1 = aux

    G_output0 = self.base.backprop_postproc(G_loss, aux0, 'select')
    G_output1 = self.base.backprop_postproc(G_loss, aux1, 'select')

    return np.hstack([G_output0, G_output1])